#### Соревнование на Kaggle

Задача:
Домашнее задание номер 3:
Коллеги подготовили для вас домашнее задание на классификацию. В рамках задания вам предлагается решить задачу и сразиться между собой в качестве. Задача на предсказание: согласится ли клиент на депозит. Найти задание можно по ссылке (для участия нужна регистрация на kaggle.com)

Нужно решить задачу с помощью 3х методов:
кнн
дерева принятия решения
логистической регрессии
и залить, на ваш взгляд, лучшее решение.

Это командное ДЗ, его нужно выполнить совместно с вашими коллегами. Коммит делается от одного выбранного участника. Для зачета нужно:
предоставить ноутбук с кодом, где вы попробовали все 3 метода
залить предсказания для теста и найти себя в лидерборде
Команда, занявшая 1 место, получит дополнительный балл. Метрика качества ROC-AUC.


Метрика ROC-AUC

* _id - идентификатор человека
* age - возраст
* job - тип работы ("admin.","blue-collar","entrepreneur","housemaid","management","retired","self-employed","services","student","technician","unemployed","unknown")
* marital - семейное положение ("divorced" (разведенный и овдовевший),"married","single","unknown")
* education - уровень образования ("basic.4y","basic.6y","basic.9y","high.school","illiterate","professional.course","university.degree","unknown")
* default - есть ли дефолт по кредиту ("no","yes","unknown")
* housing - есть ли кредит на жилье ("no","yes","unknown")
* loan - есть ли персональный кредит ("no","yes","unknown")
* contact - контактный тип связи ("cellular","telephone")
* month - последний контактный месяц года ("jan", "feb", "mar", ..., "nov", "dec")
* day_of_week - последний контактный день недели ("mon","tue","wed","thu","fri")
* duration - длительность последнего контакта в секундах. (Важное замечание: этот атрибут сильно влияет на таргет (например, если длительность - 0 сек., то таргет - 0). Тем не менее, продолжительность не известна до выполнения вызова.
* campaign - количество контактов, выполненных для этого клиента в рамках текущей маркетинговой кампании (включая последний контакт)
* pdays - количество дней прошедших последнего контакта с клиентом с предыдущей маркетинговой кампании (999 означает, что ранее с клиентом не контактировали)
* previous - количество контактов, выполненных до текущей маркетинговой кампании для этого клиента
* poutcome - результат предыдущей маркетинговой кампании ("failure","nonexistent","success")

ниже приведены показатели окружающей среды и банка

* emp.var.rate - изменение уровня занятости (квартальный показатель)
* cons.price.idx - индекс потребительских цен (ежемесячный показатель)
* cons.conf.idx - индекс доверия потребителя (ежемесячный показатель)
* euribor3m - курс Euribor (ежедневный показатель)
* nr.employed - количество сотрудников (квартальный показатель)
* target - целевая переменная (таргет) - согласился ли клиент на вклад (депозит)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split

In [2]:
# Грузим данные
row_data_train = pd.read_csv('train.csv', encoding='utf-8')
row_data_test = pd.read_csv('test.csv', encoding='utf-8')

In [3]:
# Добавляем в датафреймы столбец для дальнейшего разбиения данных
row_data_train.loc[:, 'sample'] = 'train' 
row_data_test.loc[:, 'sample'] = 'test'

In [4]:
# Склеиваем тренировочный и тестовый df и смотрим данные
row_data = row_data_train.append(row_data_test, ignore_index=True, sort=False)
row_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 23 columns):
_id               41188 non-null object
age               41188 non-null int64
job               41188 non-null object
marital           41188 non-null object
education         41188 non-null object
default           41188 non-null object
housing           41188 non-null object
loan              41188 non-null object
contact           41188 non-null object
month             41188 non-null object
day_of_week       41188 non-null object
duration          41188 non-null int64
campaign          41188 non-null int64
pdays             41188 non-null int64
previous          41188 non-null int64
poutcome          41188 non-null object
emp.var.rate      41188 non-null float64
cons.price.idx    41188 non-null float64
cons.conf.idx     41188 non-null float64
euribor3m         41188 non-null float64
nr.employed       41188 non-null float64
target            24712 non-null float64
sample    

In [5]:
# Смотрим какие стольцы типа object
for i in row_data.columns:
    if i == '_id':
        continue
    if str(row_data[i].dtype) == 'object':
        print('='*10)
        print(i)
        print(set(row_data[i]))
        print('\n')

job
{'services', 'management', 'blue-collar', 'retired', 'entrepreneur', 'housemaid', 'admin.', 'unknown', 'technician', 'unemployed', 'student', 'self-employed'}


marital
{'married', 'divorced', 'unknown', 'single'}


education
{'basic.9y', 'professional.course', 'illiterate', 'high.school', 'university.degree', 'unknown', 'basic.6y', 'basic.4y'}


default
{'yes', 'unknown', 'no'}


housing
{'yes', 'unknown', 'no'}


loan
{'yes', 'unknown', 'no'}


contact
{'cellular', 'telephone'}


month
{'jun', 'dec', 'mar', 'aug', 'nov', 'apr', 'may', 'sep', 'jul', 'oct'}


day_of_week
{'tue', 'wed', 'mon', 'fri', 'thu'}


poutcome
{'success', 'failure', 'nonexistent'}


sample
{'test', 'train'}




In [6]:
# Для атрибутов типа object используем one-hot-encoding
data = pd.get_dummies(row_data.drop(['_id', 'sample'], axis=1))
data = pd.concat([data, row_data[['_id', 'sample']]], axis=1)
data_test = data[data['sample'] == 'test'].drop(['target', 'sample'], axis=1).reset_index(drop=True)
data_train = data[data['sample'] == 'train'].drop(['sample'], axis=1).reset_index(drop=True)
features_train, features_test, targets_train, targets_test = train_test_split(data_train.drop(['target', '_id'], axis=1), data_train['target'], test_size=0.2, random_state=42)
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24712 entries, 0 to 24711
Data columns (total 65 columns):
age                              24712 non-null int64
duration                         24712 non-null int64
campaign                         24712 non-null int64
pdays                            24712 non-null int64
previous                         24712 non-null int64
emp.var.rate                     24712 non-null float64
cons.price.idx                   24712 non-null float64
cons.conf.idx                    24712 non-null float64
euribor3m                        24712 non-null float64
nr.employed                      24712 non-null float64
target                           24712 non-null float64
job_admin.                       24712 non-null uint8
job_blue-collar                  24712 non-null uint8
job_entrepreneur                 24712 non-null uint8
job_housemaid                    24712 non-null uint8
job_management                   24712 non-null uint8
job_retired    

In [8]:
# Соберем pipline для всех моделей
model_tree = DecisionTreeClassifier()
pipe_linear = Pipeline([('scaler', MinMaxScaler()), ('model', LogisticRegression())])
pipe_neighb = Pipeline([('scaler', MinMaxScaler()), ('model', KNeighborsClassifier())])

#### КНН - KNeighborsClassifier - Метод k-ближайших соседей

In [10]:
pipe_neighb = Pipeline([('scaler', MinMaxScaler()), ('model', KNeighborsClassifier())])
param_grid = {
    'model__n_neighbors': range(1,6)
}
grid_search = GridSearchCV(pipe_neighb, param_grid=param_grid, cv=5, scoring='roc_auc', n_jobs=-1)
grid_search.fit(features_train, targets_train)
print('\tНа тренеровочных данных:')
print('Результат: {:.2f}'.format(grid_search.best_score_))
print('Соответствует параметрам:\n', grid_search.best_params_)
print('\n'*2, '\tНа тестовых данных:')
print('Результат: {:.2f}'.format(grid_search.score(features_test, targets_test)))

	На тренеровочных данных:
Результат: 0.75
Соответствует параметрам:
 {'model__n_neighbors': 5}


 	На тестовых данных:
Результат: 0.78


#### DecisionTreeClassifier - дерево принятия решений

In [11]:
model_tree = DecisionTreeClassifier()
param_grid = {
    'max_depth': [2, 4, 6, 8, 10, 20, 40, 100],
}
grid_search = GridSearchCV(model_tree, param_grid=param_grid, cv=5, scoring='roc_auc', n_jobs=-1)
grid_search.fit(features_train, targets_train)
print('\tНа тренеровочных данных:')
print('Результат: {:.2f}'.format(grid_search.best_score_))
print('Соответствует параметрам:\n', grid_search.best_params_)
print('\n'*2, '\tНа тестовых данных:')
print('Результат: {:.2f}'.format(grid_search.score(features_test, targets_test)))

	На тренеровочных данных:
Результат: 0.92
Соответствует параметрам:
 {'max_depth': 6}


 	На тестовых данных:
Результат: 0.91


#### LogisticRegression - логистическая регрессия

In [12]:
pipe_linear = Pipeline([('scaler', MinMaxScaler()), ('model', LogisticRegression())])
param_grid = {
    'model__penalty': ['l1', 'l2'],
    'model__C': np.logspace(-5, 5, 11),
}
grid_search = GridSearchCV(pipe_linear, param_grid=param_grid, cv=5, scoring='roc_auc', n_jobs=-1)
grid_search.fit(features_train, targets_train)
best_model = grid_search.best_estimator_
print('\tНа тренеровочных данных:')
print('Результат: {:.2f}'.format(grid_search.best_score_))
print('Соответствует параметрам:\n', grid_search.best_params_)
print('\n'*2, '\tНа тестовых данных:')
print('Результат: {:.2f}'.format(grid_search.score(features_test, targets_test)))

	На тренеровочных данных:
Результат: 0.93
Соответствует параметрам:
 {'model__C': 100000.0, 'model__penalty': 'l1'}


 	На тестовых данных:
Результат: 0.93


#### Записываем лучшие данные

In [13]:
best_model.fit(data_train.drop(['target', '_id'], axis=1), data_train['target'])
results = best_model.predict(data_test.drop('_id', axis=1))
results = results.astype(int)
_id = data_test['_id'].values
df = pd.DataFrame(results)
df.index = _id
df.columns = ['target']
df.index.name='_id'
df.to_csv('results.csv', header=True)